In [2]:
import requests
from bs4 import BeautifulSoup
import json 
import logging
import os
from urllib.parse import urlencode
from datetime import datetime
import time

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
}

In [4]:
base_url = "https://luatvietnam.vn/tim-van-ban.html?Keywords=&SearchByDate=IssueDate&DateFrom=&DateTo=&DocTypeId=10&OrganId=0&FieldId=24&LanguageId=0&SignerId=0&SignerName=&pSize=30"

In [5]:
default_params = {
        'Keywords': '',
        'SearchByDate': 'IssueDate',
        'DateFrom': '',
        'DateTo': '',
        'DocTypeId': '10',
        'OrganId': '0',
        'FieldId': '24',
        'LanguageId': '0',
        'SignerId': '0',
        'SignerName': '',
    }

In [6]:
reponse = requests.get(base_url,headers=headers)

In [7]:
soup = BeautifulSoup(reponse.content, 'html.parser')

In [8]:
soup

<!DOCTYPE html>
<html lang="vi"><head><meta charset="utf-8"/><meta content="width=device-width, target-densitydpi=160dpi, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/><title>Tra cứu Văn bản Pháp Luật - Tìm kiếm Văn bản Luật miễn phí</title><meta content="Tra cứu văn bản pháp Luật Việt Nam, Tìm kiếm Văn bản Luật miễn phí, Công văn, Nghị định Thông tư hướng dẫn Luật đầy đủ nhất" name="description"/><meta content="Tra cứu văn bản,Tìm kiếm văn bản,Tra cứu văn bản pháp luật,Thư viện pháp luật,CSDL Luật quốc gia Việt nam,Tra cứu hiệu lực văn bản pháp luật" name="keywords"/><link href="https://luatvietnam.vn/tim-van-ban.html" rel="canonical"/><link href="/tim-van-ban.html?Keywords=&amp;SearchByDate=IssueDate&amp;DateFrom=&amp;DateTo=&amp;DocTypeId=10&amp;OrganId=0&amp;FieldId=24&amp;LanguageId=0&amp;SignerId=0&amp;SignerName=&amp;pSize=30&amp;page=2" rel="next"/><meta content="noindex, nofollow, noarchive, noodp, noydir" name="robots"/><link href="https://luatvietn

In [9]:
main_content = soup.select_one('main#site-content.main')
if not main_content:
    print("Could not find main content section")               
content_left = main_content.select_one('div.content-left')
section = content_left.select_one('section.section') if content_left else None
section_content = section.select_one('div.section-content') if section else None
block_content_shadow = section_content.select_one('div.block-content.shadow') if section_content else None
search_results = block_content_shadow.select_one('div#search_results') if block_content_shadow else None
block_content = search_results.select_one('div.block-content') if search_results else None
doc_articles = block_content.select('article.doc-article') if block_content else None

In [10]:
len(doc_articles)

25

In [11]:
documents = []
for article in doc_articles:
        # Get doc-clumn2 for each article
        doc_clumn2 = article.select_one('div.doc-clumn2')
        if doc_clumn2:
            # Get post-doc info
            post_doc = doc_clumn2.select_one('div.post-doc')
            if post_doc:
                # Get the title and link
                title_elem = post_doc.select_one('h2.doc-title a, h3.doc-title a')
                if title_elem:
                    doc_info = {
                        'href': title_elem.get('href'),
                        'title': title_elem.text.strip(),
                        'number': article.select_one('span.doc-number').text.strip() if article.select_one('span.doc-number') else None
                    }
                    
                    # Get dates if available
                    dates = {}
                    date_elements = doc_clumn2.select('div.doc-dmy')
                    for date_elem in date_elements:
                        label = date_elem.select_one('span.w-doc-dmy1')
                        value = date_elem.select_one('span.w-doc-dmy2')
                        if label and value:
                            key = label.text.strip().replace(':', '')
                            dates[key] = value.text.strip()
                    
                    doc_info['dates'] = dates
                    documents.append(doc_info)

In [12]:
herf = documents[1]['href']
tem_url = "https://luatvietnam.vn" + herf
reponse = requests.get(tem_url, headers=headers)
soup = BeautifulSoup(reponse.content, 'html.parser')
main = soup.select_one('main.main')
if not main:
    print("Could not find main content section")  
main_content = main.select_one('div.main-content')
wrapper = main_content.select_one('div.wrapper')
content_left = wrapper.select_one('div.content-left')
document = content_left.select_one('article.the-document')
tab_content = document.select_one('div.tab-content.fix-border')
tab_noi_dung = tab_content.select_one('div#chidanthaydoi')


In [13]:
import re
import json
from datetime import datetime
import logging

# Cấu hình logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('law_parser.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

result = {
    "title": "",
    "id_doc": "",
    "location": "Hà Nội",
    "doc_type": "Doanh Nghiep",
    "date": "",
    "base": [],
    "signs": [],
    "data": {},
    "department": "Quốc hội",
    "doc_type_en": "The law"
}

def parse_law_structure(docitems):
    signs_text = []
    date_found = False
    all_content = []
    
    logging.info(f"Bắt đầu parse với số lượng docitems: {len(docitems)}")
    
    for item in docitems:
        text = item.get_text(strip=True)
        
        # Xử lý phần chữ ký và ngày tháng
        if "Luật này được Quốc hội" in text or "CHỦ TỊCH" in text:
            date_found = True
            signs_text.append(text)
            continue
            
        if date_found:
            signs_text.append(text)
            continue

        # Collect all content
        if text:
            all_content.append(text)
    
    # Save all content to data
    result['data'] = {
        "content": "\n".join(all_content)
    }

    # Xử lý phần chữ ký
    if signs_text:
        full_signs = ' '.join(signs_text)
        
        name_match = re.search(r'Đã ký:?\s*([^.]+)', full_signs)
        if name_match:
            name = name_match.group(1).strip()
            result["signs"] = [name]
            
        date_match = re.search(r'Luật này được.*?thông qua ngày (\d+) tháng (\d+) năm (\d+)', full_signs)
        if date_match:
            ngay, thang, nam = date_match.groups()
            result["date"] = f"{ngay}/{thang}/{nam}"



In [14]:
documents

[{'href': '/doanh-nghiep/luat-hop-tac-xa-moi-nhat-2023-so-17-2023-qh15-259729-d1.html',
  'title': 'Luật Hợp tác xã, số 17/2023/QH15',
  'number': '01',
  'dates': {'Ban hành': '20/06/2023',
   'Áp dụng': 'Đã biếtVui lòng đăng nhập tài khoản gói Tiêu chuẩn hoặc Nâng cao để xem ngày Áp dụng. Nếu chưa có tài khoản Quý khách đăng ký tại đây!',
   'Hiệu lực': 'Đã biếtVui lòng đăng nhập tài khoản gói Tiêu chuẩn hoặc Nâng cao để xem Hiệu lực. Nếu chưa có tài khoản Quý khách đăng ký tại đây!',
   'Cập nhật': '18/07/2023'}},
 {'href': '/dau-tu/luat-sua-doi-luat-dau-tu-cong-luat-dau-tu-theo-phuong-thuc-doi-tac-cong-tu-2022-216275-d1.html',
  'title': 'Luật sửa đổi Luật Đầu tư công, Luật Đầu tư theo phương thức đối tác công tư, Luật Đầu tư, Luật Nhà ở, Luật Đấu thầu, Luật Điện lực, Luật Doanh nghiệp, Luật Thuế tiêu thụ đặc biệt, Luật Thi hành án dân sự',
  'number': '02',
  'dates': {'Ban hành': '11/01/2022',
   'Áp dụng': 'Đã biếtVui lòng đăng nhập tài khoản gói Tiêu chuẩn hoặc Nâng cao để xem 

In [15]:
# for i in range(0,25):
#     herf = documents[i]['href']
#     tem_url = "https://luatvietnam.vn" + herf
#     reponse = requests.get(tem_url, headers=headers)
#     soup = BeautifulSoup(reponse.content, 'html.parser')
#     main = soup.select_one('main.main')
#     if not main:
#         print("Could not find main content section")  
#     main_content = main.select_one('div.main-content')
#     wrapper = main_content.select_one('div.wrapper')
#     content_left = wrapper.select_one('div.content-left')
#     document = content_left.select_one('article.the-document')
#     tab_content = document.select_one('div.tab-content.fix-border')
#     tab_noi_dung = tab_content.select_one('div#chidanthaydoi')

    
#     # Xử lý title và id_doc
#     data_title = tab_content.get('data-title')
#     if data_title:
#         match = re.search(r'\b(\d{2}/\d{4}/QH\d{2})\b', data_title)
#         title_part = data_title.split(', số')[0].strip()
#         result["title"] = title_part
#         if match:
#             document_number = match.group(1)
#             result["id_doc"] = document_number

#     # Parse docitems
#     docitems = tab_noi_dung.find_all("div", class_=lambda x: x and x.startswith("docitem"))
#     parse_law_structure(docitems)

#     # Save to JSON file
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
#     filename = f"{result['title'].replace(' ', '_')}.json"
#     special = "Điều_chỉnh_luật.json"
#     if "_số" in filename:
#         clean_name = re.sub(r'_số.*\.json$', '.json', filename)
#         filename = clean_name


#     try:
#         with open(filename, 'w', encoding='utf-8') as f:
#             json.dump(result, f, ensure_ascii=False, indent=2)
#         print(f"Đã lưu thành công vào file: {filename}")
#     except Exception as e:
#         print(f"Lỗi khi lưu file: {str(e)}")


In [16]:
tem_url = "https://luatvietnam.vn/giao-thong/luat-sua-dong-hang-khong-91373-d1.html"
reponse = requests.get(tem_url, headers=headers)
soup = BeautifulSoup(reponse.content, 'html.parser')
main = soup.select_one('main.main')
if not main:
    print("Could not find main content section")  
main_content = main.select_one('div.main-content')
wrapper = main_content.select_one('div.wrapper')
content_left = wrapper.select_one('div.content-left')
document = content_left.select_one('article.the-document')
tab_content = document.select_one('div.tab-content.fix-border')
tab_noi_dung = tab_content.select_one('div#chidanthaydoi')

    
# Xử lý title và id_doc
data_title = tab_content.get('data-title')
if data_title:
    match = re.search(r'\b(\d{2}/\d{4}/QH\d{2})\b', data_title)
    title_part = data_title.split(', số')[0].strip()
    result["title"] = title_part
    if match:
        document_number = match.group(1)
        result["id_doc"] = document_number

    # Parse docitems
    docitems = tab_noi_dung.find_all("div", class_=lambda x: x and x.startswith("docitem"))
    parse_law_structure(docitems)

    # Save to JSON file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{result['title'].replace(' ', '_')}.json"
    special = "Điều_chỉnh_luật.json"
    if "_số" in filename:
        clean_name = re.sub(r'_số.*\.json$', '.json', filename)
        filename = clean_name


    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=2)
        print(f"Đã lưu thành công vào file: {filename}")
    except Exception as e:
        print(f"Lỗi khi lưu file: {str(e)}")

2024-11-06 22:05:20,597 - DEBUG - Starting new HTTPS connection (1): luatvietnam.vn:443
2024-11-06 22:05:20,760 - DEBUG - https://luatvietnam.vn:443 "GET /giao-thong/luat-sua-dong-hang-khong-91373-d1.html HTTP/11" 200 None
2024-11-06 22:05:20,841 - INFO - Bắt đầu parse với số lượng docitems: 109


Đã lưu thành công vào file: Luật_sửa_đổi,_bổ_sung_một.json
